In [1]:
# Importing necessities

import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf
import cv2
import os
import posenet
import time
import pandas as pd
import numpy as np


In [2]:
# Directory of videos to be considered
dir_ = ".\data"

files = os.listdir(dir_)
#file = files[0]
print(files)
os.path.join(dir_, files[0])

['bhavesh_ingale_pushups.mp4', 'bhavesh_ingale_sit.mp4', 'bhavesh_ingale_walking-with-hands-up.mp4', 'bhavesh_ingale_walking.mp4', 'edward_praveen_pushups.mp4', 'online_fall.mp4', 'online_pushups.mp4', 'online_sit.mp4', 'online_walking.mp4', 'rahul_gupta_actions1.mp4']


'.\\data\\bhavesh_ingale_pushups.mp4'

In [3]:
# Configuring my cudnn for tensorflow 1
# Needed for my laptop, unsure for others

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

```python
csv_name = 'my_csv.csv'
with tf.Session() as sess:
    
    # Setting ouput stride to 16
    output_stride = 16
    model_id = 101
    model_cfg, model_outputs = posenet.load_model(model_id, sess)
    
    # change to for loop after testing on single file
    file = files[0]
    action = files[0].split("_")[-1].split(".")[0]
    cap = cv2.VideoCapture(os.path.join(dir_, file))
    input_image, display_image, output_scale = posenet.read_cap(cap)
    
    start = time.time()
    frame_count = 0
    while True:
        
        input_image, display_image, output_scale = posenet.read_cap(cap)

        heatmaps_result, offsets_result, displacement_fwd_result, displacement_bwd_result = sess.run(
            model_outputs,
            feed_dict={'image:0': input_image})

        # print(heatmaps_result.shape) # (1, 33, 58, 17)

        pose_scores, keypoint_scores, keypoint_coords = posenet.decode_multi.decode_multiple_poses(
            heatmaps_result.squeeze(axis=0),
            offsets_result.squeeze(axis=0),
            displacement_fwd_result.squeeze(axis=0),
            displacement_bwd_result.squeeze(axis=0),
            output_stride=output_stride,
            max_pose_detections=10,
            min_pose_score=0.15)

        keypoint_coords *= output_scale

        # TODO this isn't particularly fast, use GL for drawing and display someday...
        overlay_image = posenet.draw_skel_and_kp(
            display_image, pose_scores, keypoint_scores, keypoint_coords,
            min_pose_score=0.15, min_part_score=0.3)
        # setting min_pose_score and min_part_score =0 prints the estimated value

        x = keypoint_coords[0][:,0]
        y = keypoint_coords[0][:,1]
        data = {'action': action, 
                'frame_number': frame_count+1, 
                'input_number': np.arange(1, posenet.NUM_KEYPOINTS+1), 
                'x_inputs': x, 'y_inputs': y}
        temp = pd.DataFrame(data=data)
        
        if csv_name not in os.listdir():
            temp.to_csv(csv_name, 
                        header=True, 
                        index=False)
        else:
            temp.to_csv(csv_name, 
                        mode='a', 
                        header=False, 
                        index=False)
        
        cv2.imshow('posenet', overlay_image)
        frame_count += 1
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # print(cap.get(cv2.CAP_PROP_FPS)) -> frame rate
    print('Average FPS: ', frame_count / (time.time() - start))
    print(frame_count)```


### Since not all videos from drives are to be taken into account, I'll download them manually

In [4]:
csv_name = 'entire_data.csv'
with tf.Session() as sess:
    for file in files:
        # Setting ouput stride to 16
        output_stride = 16
        model_id = 101
        model_cfg, model_outputs = posenet.load_model(model_id, sess)

        # change to for loop after testing on single file
        #file = files[0]
        action = file.split("_")[-1].split(".")[0]
        cap = cv2.VideoCapture(os.path.join(dir_, file))
        input_image, display_image, output_scale = posenet.read_cap(cap)

        start = time.time()
        frame_count = 0
        check = True
        while check:
            try:
                input_image, display_image, output_scale = posenet.read_cap(cap)

                heatmaps_result, offsets_result, displacement_fwd_result, displacement_bwd_result = sess.run(
                    model_outputs,
                    feed_dict={'image:0': input_image})

                # print(heatmaps_result.shape) # (1, 33, 58, 17)

                pose_scores, keypoint_scores, keypoint_coords = posenet.decode_multi.decode_multiple_poses(
                    heatmaps_result.squeeze(axis=0),
                    offsets_result.squeeze(axis=0),
                    displacement_fwd_result.squeeze(axis=0),
                    displacement_bwd_result.squeeze(axis=0),
                    output_stride=output_stride,
                    max_pose_detections=10,
                    min_pose_score=0.15)

                keypoint_coords *= output_scale

                # TODO this isn't particularly fast, use GL for drawing and display someday...
                overlay_image = posenet.draw_skel_and_kp(
                    display_image, pose_scores, keypoint_scores, keypoint_coords,
                    min_pose_score=0.15, min_part_score=0.3)
                # setting min_pose_score and min_part_score =0 prints the estimated value

                x = keypoint_coords[0][:,0]
                y = keypoint_coords[0][:,1]
                data = {'action': action, 
                        'frame_number': frame_count+1, 
                        'input_number': np.arange(1, posenet.NUM_KEYPOINTS+1), 
                        'x_inputs': x, 'y_inputs': y}
                temp = pd.DataFrame(data=data)

                if csv_name not in os.listdir():
                    temp.to_csv(csv_name, 
                                header=True, 
                                index=False)
                else:
                    temp.to_csv(csv_name, 
                                mode='a', 
                                header=False, 
                                index=False)

                #cv2.imshow('posenet', overlay_image)
                frame_count += 1
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            except:
                print(f"Completed Writing {file} data")
                cap.release()
                check = False
            
    # print(cap.get(cv2.CAP_PROP_FPS)) -> frame rate
    #print('Average FPS: ', frame_count / (time.time() - start))
    #print(frame_count)
    print(f'time taken = {(time.time() - start)/60} mins')




Completed Writing bhavesh_ingale_pushups.mp4 data
Completed Writing bhavesh_ingale_sit.mp4 data
Completed Writing bhavesh_ingale_walking-with-hands-up.mp4 data
Completed Writing bhavesh_ingale_walking.mp4 data
Completed Writing edward_praveen_pushups.mp4 data
Completed Writing online_fall.mp4 data
Completed Writing online_pushups.mp4 data
Completed Writing online_sit.mp4 data
Completed Writing online_walking.mp4 data
Completed Writing rahul_gupta_actions1.mp4 data
time taken = 0.5766624410947164 mins
